<a href="https://colab.research.google.com/github/youngmoo/ECES-434/blob/main/Class%209.1%20(2021-03-08).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-434: Class 9.1 (2021-03-08)**
Week 9: Last week of classes!! 😱



In [ ]:
import numpy as np                      # NumPy, abbreviated as np
import matplotlib.pyplot as plt         # MatplotLib PyPlot module, abbreviated as plt
from matplotlib import animation, rc    # MatplotLib animation module
%matplotlib inline
from scipy import signal                # SciPy's signal module, for DSP functions
import soundfile as sf                  # Switching to the soundfile module for reading and writing soundfiles
import librosa

import IPython.display as ipd           # Interactive Python display module, for playing sounds
from IPython.display import HTML        # For displaying animations
rc('animation', html='jshtml')          # Provides animation controls

ClassPath = '/content/drive/My Drive/ECES-434 Sessions/Class 9-1/'


In [ ]:
rc('figure', figsize=(16,4))

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'anonymous'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Custom plotting functions
Because we're always plotting...

## plotSpectrogram

In [ ]:
def plotSpectrogram(sig, fs, win='hann', nseg=512, olap=256, fft_len=512):
  
  f1, t1, Sxx = signal.spectrogram(sig, fs, window=win, nperseg=nseg, noverlap=olap, nfft=fft_len)
  fig = plt.figure(figsize=(16,6))
  
  plt.pcolormesh(t1, f1, 20*np.log10(np.abs(Sxx)))
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')
  return plt

## myPlot(): properly formats time domain plot of a signal

In [ ]:
def myPlot(sig, fs=44100):
  fig = plt.figure(figsize=(16,4))
  t = np.arange(len(sig)) / fs
  plt.plot(t, sig)
  plt.xlabel('Time (sec)')
  return fig, plt

## myPlotFFT(): properly formats frequency domain plot of a signal

In [ ]:
def myPlotFFT(sig, n_fft=0, x_lim=22050, fs=44100):
  if n_fft==0:                 
    n_fft = len(sig)                    # Default to length of input signal
  S = np.fft.fft(sig, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  fig = plt.figure(figsize=(16,6))
  plt.plot(f, 20*np.log10(np.abs(S)))
  plt.xlim(0, x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB)')
  return fig, plt  

## myPlotFFTPhase

In [ ]:
def myPlotFFTPhase(sig, n_fft=0, x_lim=22050, fs=44100):
  if n_fft==0:                 
    n_fft = len(sig)                    # Default to length of input signal
  S = np.fft.fft(sig, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  fig = plt.figure(figsize=(16,4))
  plt.plot(f, np.unwrap(np.angle(S)))
  plt.xlim(0, x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Phase (radians)')
  return fig, plt

## Custom FFT animation functions

In [ ]:
n_o = 0
f_size = 2048
n_hop = f_size / 2
N_fft = 4096
fs = 44100
this_f = np.arange(N_fft) * fs / N_fft

# First set up the figure, the axis, and the plot element we want to animate
def setupAnimFFT(x_lim=(0,20000), y_lim=(-120,100)):
  fig = plt.figure(figsize=(14,6))
  ax = plt.axes(xlim=x_lim,ylim=y_lim)
  plt.close()   # Don't output the final figure separately
  line, = ax.plot([], [])
  fig.tight_layout()
  return fig, line

# initialization function: plot the background of each frame
def initAnimFFT():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially  
def animateFFT(i, sig):
    n1 = int(n_o + n_hop*i)
    n2 = int(n_o + n_hop*i + f_size)

    x_i = sig[n1:n2]
    X_i = np.fft.fft(x_i * np.hanning(len(x_i)), n=N_fft)
    X_mag = 20*np.log(np.abs(X_i))

    line.set_data(this_f, X_mag)
    return (line,)  

# Usage:
# fig, line = setupAnimFFT()
# anim = animation.FuncAnimation(fig, animateFFT, init_func=initAnimFFT, frames=120, fargs=(signal,), interval=1000/30, blit=True)
# anim

# Displaying tablular data

In [ ]:
import tabulate

table = [["Object", "Data 1", "Data 2"],
         ["Sun", 696000, 1989100000],
         ["Earth", 6371, 5973.6],
         ["Moon", 1737, 73.5],
         ["Mars", 3390, 641.85]]

print(tabulate.tabulate(table, headers='firstrow'))

In [ ]:
ipd.HTML(tabulate.tabulate(table, headers='firstrow', tablefmt='html'))

# Codecs used by streaming services


* [Streaming codec comparison](https://rodneyorpheus.medium.com/streaming-audio-quality-overview-e5eb8e79dd2) 
* [Spotify info](https://support.spotify.com/us/article/high-quality-streaming/)



# Load sound file
Closing out the year with U2: 
*Mysterious Ways* (1991)

In [ ]:
u2, fs = sf.read(ClassPath + 'MysteriousWays-44.1kHz.wav')
ipd.Audio(u2.T,rate=fs)

In [ ]:
u2_flac, fs = sf.read(ClassPath + 'MysteriousWays.flac')
ipd.Audio(u2_flac.T,rate=fs)
plt.plot(u2 - u2_flac)

In [ ]:
u2_ogg320, fs = sf.read(ClassPath + 'MysteriousWays-320kbps.ogg')
ipd.Audio(u2_ogg320.T,rate=fs)

In [ ]:
plt.plot(u2 - u2_ogg320)
ipd.Audio(u2.T - u2_ogg320.T, rate=fs)

In [ ]:
t = np.arange(len(u2))/fs
plt.plot(t,u2[:,0],t,u2_ogg320[:,0])
plt.xlim(0.1,0.11)

In [ ]:
sqerr_ogg320 = (u2 - u2_ogg320) ** 2 
snr_ogg320 = np.sum(u2 ** 2, axis=0) / np.sum(sqerr_ogg320, axis=0)
print( 20*np.log10(snr_ogg320) )

In [ ]:
u2_aac256, fs = sf.read(ClassPath + 'MysteriousWays-AAC256kbps.wav')
ipd.Audio(u2_aac256.T,rate=fs)

In [ ]:
plt.plot(u2 - u2_aac256)
ipd.Audio(u2.T - u2_aac256.T, rate=fs)

In [ ]:
sqerr_aac256 = (u2 - u2_aac256) ** 2 
snr_aac256 = np.sum(u2 ** 2, axis=0) / np.sum(sqerr_aac256, axis=0)
print( 20*np.log10(snr_aac256) )

In [ ]:
u2_aac128, fs = sf.read(ClassPath + 'MysteriousWays-AAC128kbps.wav')
ipd.Audio(u2_aac128.T,rate=fs)

In [ ]:
plt.plot(u2 - u2_aac128)
ipd.Audio(u2.T - u2_aac128.T, rate=fs)

In [ ]:
t = np.arange(len(u2))/fs
plt.plot(t,u2[:,0],t,u2_aac128[:,0])
#plt.xlim(0.1,0.11)